In [30]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt


In [31]:
Mental_Health_Twitter_data=pd.read_csv('Mental-Health-Twitter.csv')
Mental_Health_Twitter_data.head(20)

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
5,5,637692793083817985,Sat Aug 29 18:26:24 +0000 2015,Making packing lists is my new hobby... #movin...,1013187241,84,211,251,837,1,1
6,6,637691649943072772,Sat Aug 29 18:21:51 +0000 2015,At what point does keeping stuff for nostalgic...,1013187241,84,211,251,837,1,1
7,7,637689418472652800,Sat Aug 29 18:12:59 +0000 2015,Currently in the finding-boxes-of-random-shit ...,1013187241,84,211,251,837,0,1
8,8,637687177946734592,Sat Aug 29 18:04:05 +0000 2015,"Can't be bothered to cook, take away on the wa...",1013187241,84,211,251,837,0,1
9,9,637684866906255360,Sat Aug 29 17:54:54 +0000 2015,RT @itventsnews: ITV releases promo video for ...,1013187241,84,211,251,837,41,1


In [32]:
Mental_Health_Twitter_data=Mental_Health_Twitter_data.drop("Unnamed: 0",axis=1)

In [33]:
Mental_Health_Twitter_data.shape

(20000, 10)

In [34]:
Mental_Health_Twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       20000 non-null  int64 
 1   post_created  20000 non-null  object
 2   post_text     20000 non-null  object
 3   user_id       20000 non-null  int64 
 4   followers     20000 non-null  int64 
 5   friends       20000 non-null  int64 
 6   favourites    20000 non-null  int64 
 7   statuses      20000 non-null  int64 
 8   retweets      20000 non-null  int64 
 9   label         20000 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 1.5+ MB


In [35]:
Mental_Health_Twitter_data.isnull()

,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,False,False,False,False,False,False


In [36]:
Mental_Health_Twitter_data.isnull().sum()

post_id         0
post_created    0
post_text       0
user_id         0
followers       0
friends         0
favourites      0
statuses        0
retweets        0
label           0
dtype: int64

In [37]:
Mental_Health_Twitter_data=Mental_Health_Twitter_data.drop("post_id",axis=1)
#Mental_Health_Twitter_data=Mental_Health_Twitter_data.drop("post_created",axis=1)
Mental_Health_Twitter_data=Mental_Health_Twitter_data.drop("user_id",axis=1)

In [38]:
Mental_Health_Twitter_data.head()

,post_created,post_text,followers,friends,favourites,statuses,retweets,label
0,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,84,211,251,837,0,1
1,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",84,211,251,837,1,1
2,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,84,211,251,837,0,1
3,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,84,211,251,837,2,1
4,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,84,211,251,837,1,1


In [39]:
Mental_Health_Twitter_data

,post_created,post_text,followers,friends,favourites,statuses,retweets,label
0,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,84,211,251,837,0,1
1,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",84,211,251,837,1,1
2,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,84,211,251,837,0,1
3,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,84,211,251,837,2,1
4,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...
19995,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,442,230,7,1063601,0,0
19996,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",442,230,7,1063601,0,0
19997,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,442,230,7,1063601,0,0
19998,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",442,230,7,1063601,0,0


In [40]:
Mental_Health_Twitter_data.corr()

,followers,friends,favourites,statuses,retweets,label
followers,1.000000,0.891919,-0.016242,-0.000215,0.001500,-0.011181
friends,0.891919,1.000000,-0.081756,-0.045420,0.009946,0.119818
favourites,-0.016242,-0.081756,1.000000,-0.053930,0.069551,-0.252289
statuses,-0.000215,-0.045420,-0.053930,1.000000,-0.005193,-0.246473
retweets,0.001500,0.009946,0.069551,-0.005193,1.000000,-0.025198
label,-0.011181,0.119818,-0.252289,-0.246473,-0.025198,1.000000


In [44]:
Mental_Health_Twitter_data.posr_created=Mental_Health_Twitter_data.post_created.apply(pd.to_datetime)
Mental_Health_Twitter_data["Month"] = Mental_Health_Twitter_data.post_created.dt.month
#Mental_Health_Twitter_data["Year"]= Mental_Health_Twitter_data.post_created.dt.year
#Mental_Health_Twitter_data["day"]= Mental_Health_Twitter_data.post_created.dt.day

AttributeError: Can only use .dt accessor with datetimelike values